## Теоретическая часть

1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?


User-User - пытается идентифицировать пользователей с наиболее похожим «профилем взаимодействия» (ближайшими соседями), чтобы предложить элементы, которые являются наиболее популярными среди этих соседей. Item-Item состоит в том, чтобы находить элементы, похожие на те, с которыми пользователь уже «положительно» взаимодействовал. У item-item алгоритмов большая сложность predict и бОльшее время обучения по сравнению с User-User. User-User - высокая дисперсия, низкий уклон. Item-Item - более низкая дисперсия, менее персонализирован.
Функция item_item_rec может использоваться для user_user_rec. Для этого в нее нужно передать не транспонированную исходную матрицу. item-item рекомендаций применяются для explicity, а ALS для inplicity. 
В основу item-item рекомендаций заложен алгоритм "К-ближайших", а в ALS - алгоритм Градиентного спуска. ALS - позволяет сэкономить место путем разбиения матрицы users-items на 2 матрицы

2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


Общий рейтинг (пользовательский) товара, среднее время нахождения в корзине до покупки, маржа с продажи

3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


Учитывает только линейные зависимости, не учитывает контекст

4. Мы рассматривали bm_25_weight. 
Опишите, как он работает. Как сделать рекомендации только на основе bm_25? (Можно и нужно пользоваться любыми источниками, приложите на них ссылки). Какие еще способы перевзвешивания user-item матрицы Вы знаете / можете предложить (ещё 2-3 способа)?

Это функция ранжирования, которая используется системами для распределения документов по их релевантности относительно запроса. Работает на основе TF-IDF. В формуле оценки релевантности документа запросу IDF имеет следующий недостаток: для слов, входящих в более чем половину документов из коллекции, значение IDF отрицательно. Таким образом, при наличии любых двух почти идентичных документов, в одном из которых есть слово, а в другом — нет, второй может получить бо́льшую оценку. 
https://ru.wikipedia.org/wiki/Okapi_BM25

## Практическая часть


In [1]:
import implicit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
import time

def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
        
    return precision

In [2]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result_for_train = data_train.groupby('user_id')['item_id'].unique().reset_index()
result_for_train.columns=['user_id', 'actual']

result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [6]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Coolver\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
csr_mat = csr_matrix(user_item_matrix).T.tocsr()

In [9]:
%%time

model = AlternatingLeastSquares(factors=100, #k
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 2.54 s


In [10]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [12]:
train = []
test = []

In [13]:
%%time

result['als_quantity_test'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
pr = result.apply(lambda row: precision_at_k(row['als_quantity_test'], row['actual']), axis=1).mean()
test.append(pr)

Wall time: 33.6 s


In [14]:
%%time 
result_for_train['als_quantity_train'] = result_for_train['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
pr = result_for_train.apply(lambda row: precision_at_k(row['als_quantity_train'], row['actual']), axis=1).mean()
train.append(pr)

Wall time: 35.9 s


In [15]:
result.head(2)

,user_id,actual,als_quantity_test
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1033142, 979707, 995242, 1029743, 1082212]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1133018, 938700, 951590, 5569327]"


### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


In [20]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='trans_time', # Можно пробоват ьдругие варианты
                                  aggfunc='sum', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

csr_mat = csr_matrix(user_item_matrix).T.tocsr()

model = AlternatingLeastSquares(factors=100, #k
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

In [21]:
result['als_trans_time_s_test'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
pr = result.apply(lambda row: precision_at_k(row['als_trans_time_s_test'], row['actual']), axis=1).mean()
test.append(pr)

result_for_train['als_trans_time_s_train'] = result_for_train['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
pr = result_for_train.apply(lambda row: precision_at_k(row['als_trans_time_s_train'], row['actual']), axis=1).mean()
train.append(pr)

In [45]:
approach = ['als_quantity', 'als_sales_value_sum', 'als_sales_value_count', 'trans_time_sum']

scores_2 = pd.DataFrame(list(zip(approach, train, test)), columns=['approach','train', 'test'])
#scores_2.set_index('approach')
scores_2

,approach,train,test
0,als_quantity,0.736855,0.160333
1,als_sales_value_sum,0.589276,0.099510
2,als_sales_value_count,0.736935,0.157689
3,trans_time_sum,0.043537,0.005583


Как и предполагалось, единственный вес, который может быть использован в нашем случае это количество товаров. Т.к. этот параметр напрямую количественно выражает взаимодействие пользователя с товаром. Сумма покупки не так хорошо сработала, полагаю, это связано с тем, что стоимость товара никак не отображает его нужность.

### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [23]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

csr_mat = csr_matrix(user_item_matrix).T.tocsr()

In [33]:
precisions_test = []
factor_n = []
lambda_1 = []

In [34]:
for factor in [100, 150, 300]:
    for lambda_ in [1, 0.01, 0.001]:
        factor_n.append(factor)
        lambda_1.append(lambda_)
        
        model = AlternatingLeastSquares(factors=factor, 
                            regularization=lambda_,
                            iterations=15, 
                            calculate_training_loss=True, 
                            num_threads=4) # K - кол-во билжайших соседей

        model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                    show_progress=True)
            
        result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        pr = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
        
        precisions_test.append(pr)


In [37]:
scores = pd.DataFrame(list(zip(precisions_test, factor_n, lambda_1)), columns=['als_test_score','factor_n', 'lambda'])
scores

,als_test_score,factor_n,lambda
0,0.162684,100,1.000
1,0.165132,100,0.010
2,0.162292,100,0.001
3,0.152987,150,1.000
4,0.153869,150,0.010
5,0.146033,150,0.001
6,0.124878,300,1.000
7,0.129971,300,0.010
8,0.117042,300,0.001


In [78]:
model = AlternatingLeastSquares(factors=5, #k
                                regularization=0.01,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

In [79]:
result['als_tunning_ts'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
pr_ts = result.apply(lambda row: precision_at_k(row['als_tunning_ts'], row['actual']), axis=1).mean()

result_for_train['als_tunning_tr'] = result_for_train['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
pr_tr = result_for_train.apply(lambda row: precision_at_k(row['als_tunning_tr'], row['actual']), axis=1).mean()

In [80]:
score = pd.DataFrame({'approach': 'als_tunning', 'train': [pr_tr], 'test': [pr_ts]})
score

,approach,train,test
0,als_tunning,0.517887,0.172674


Увеличивая factor_n снижается качество предсказаний. Подбор параметров работает очень долго, поэтому завершающий тюннинг делала вручную, снижая число factor_n. За счет регуляризации (лямбды) мы немного снизили разрыв между результатами трэйна и теста.

**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!